## Librerias

In [31]:
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
from glob import glob
import pandas as pd
import numpy as np
import os
import re

# Datos de dengue

In [32]:
entidades_federativas = {
    1: "Aguascalientes",
    2: "Baja California",
    3: "Baja California Sur",
    4: "Campeche",
    5: "Coahuila",
    6: "Colima",
    7: "Chiapas",
    8: "Chihuahua",
    9: "Ciudad de México",
    10: "Durango",
    11: "Guanajuato",
    12: "Guerrero",
    13: "Hidalgo",
    14: "Jalisco",
    15: "Estado de México",
    16: "Michoacán",
    17: "Morelos",
    18: "Nayarit",
    19: "Nuevo León",
    20: "Oaxaca",
    21: "Puebla",
    22: "Querétaro",
    23: "Quintana Roo",
    24: "San Luis Potosí",
    25: "Sinaloa",
    26: "Sonora",
    27: "Tabasco",
    28: "Tamaulipas",
    29: "Tlaxcala",
    30: "Veracruz",
    31: "Yucatán",
    32: "Zacatecas"
}

## Carga de datos

In [33]:
files_degue = glob('data/dengue/datos_abiertos_dengu*/*.csv')
files_degue

['data/dengue\\datos_abiertos_dengue_230323\\dengue_abierto.csv',
 'data/dengue\\datos_abiertos_dengue_291222\\dengue_abierto.csv',
 'data/dengue\\datos_abiertos_dengue_301221\\Datos abiertos dengue_301221.csv',
 'data/dengue\\datos_abiertos_dengue_311220\\Datos abiertos dengue_311220.csv']

In [34]:
dengue = pd.DataFrame()
for f in files_degue:
    df = pd.read_csv(f)
    print(df.shape)
    dengue = pd.concat([dengue, df], axis=0)
print(dengue.shape)
dengue.head()

(9439, 28)
(57730, 28)
(35413, 28)
(120239, 28)
(222821, 28)


,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,INMUNOSUPR,CIRROSIS_HEPATICA,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG
0,2023-03-21,1007423,2,29,23,5,2,2,23,5,...,2,2,2,2,5.0,1,5,3,23,5
1,2023-03-21,1007433,2,25,27,4,2,2,27,4,...,2,2,2,2,5.0,1,5,3,27,4
2,2023-03-21,1007459,2,0,7,89,2,2,7,89,...,2,2,2,2,5.0,2,5,1,7,89
3,2023-03-21,1007452,2,41,12,22,2,2,12,22,...,2,2,2,2,5.0,2,5,1,12,22
4,2023-03-21,1007468,2,10,12,73,2,2,12,73,...,2,2,2,2,5.0,1,5,1,12,73


In [35]:
dengue['FECHA_SIGN_SINTOMAS'] = pd.to_datetime(dengue['FECHA_SIGN_SINTOMAS'], format='%Y-%m-%d')

In [36]:
dengue['ENTIDAD_RES'] = dengue['ENTIDAD_RES'].map(entidades_federativas)
dengue['ENTIDAD_UM_NOTIF'] = dengue['ENTIDAD_UM_NOTIF'].map(entidades_federativas)
dengue['ENTIDAD_ASIG'] = dengue['ENTIDAD_ASIG'].map(entidades_federativas)

In [37]:
dengue = dengue[dengue['ESTATUS_CASO'] != 3]
dengue

,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,INMUNOSUPR,CIRROSIS_HEPATICA,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG
2,2023-03-21,1007459,2,0,Chiapas,89,2,2,Chiapas,89,...,2,2,2,2,5.0,2,5,1,Chiapas,89
3,2023-03-21,1007452,2,41,Guerrero,22,2,2,Guerrero,22,...,2,2,2,2,5.0,2,5,1,Guerrero,22
4,2023-03-21,1007468,2,10,Guerrero,73,2,2,Guerrero,73,...,2,2,2,2,5.0,1,5,1,Guerrero,73
6,2023-03-21,1007475,1,12,Yucatán,79,2,2,Yucatán,79,...,2,2,2,2,5.0,2,5,1,Yucatán,79
7,2023-03-21,1007462,1,12,Guerrero,35,2,2,Guerrero,35,...,2,2,2,2,5.0,1,5,2,Guerrero,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120234,2020-12-30,841970,1,40,Morelos,25,2,2,Morelos,12,...,2,2,2,2,5.0,1,5,1,Morelos,25
120235,2020-12-30,841973,2,53,Jalisco,97,2,2,Jalisco,98,...,2,2,2,2,5.0,2,5,1,Jalisco,97
120236,2020-12-30,841976,1,31,Nuevo León,4,2,2,Nuevo León,4,...,2,2,2,2,5.0,1,2,2,Nuevo León,4
120237,2020-12-30,841979,2,28,Nuevo León,6,2,2,Nuevo León,6,...,2,2,2,2,5.0,1,5,1,Nuevo León,6


### Dengue por entidad federativa

In [38]:
dengue.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'SEXO', 'EDAD_ANOS',
       'ENTIDAD_RES', 'MUNICIPIO_RES', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'ENTIDAD_UM_NOTIF', 'MUNICIPIO_UM_NOTIF', 'INSTITUCION_UM_NOTIF',
       'FECHA_SIGN_SINTOMAS', 'TIPO_PACIENTE', 'HEMORRAGICOS', 'DIABETES',
       'HIPERTENSION', 'ENFERMEDAD_ULC_PEPTICA', 'ENFERMEDAD_RENAL',
       'INMUNOSUPR', 'CIRROSIS_HEPATICA', 'EMBARAZO', 'DEFUNCION', 'DICTAMEN',
       'TOMA_MUESTRA', 'RESULTADO_PCR', 'ESTATUS_CASO', 'ENTIDAD_ASIG',
       'MUNICIPIO_ASIG'],
      dtype='object')

Se mantendrán los registros que sean DICTAMEN = 1 y 4, Confirmado y En estudio, respectivamente.

In [39]:
dengue = dengue[dengue['DICTAMEN'].isin([1,4])]
dengue

,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,INMUNOSUPR,CIRROSIS_HEPATICA,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG
1171,2023-03-21,1002287,1,57,Tamaulipas,3,2,2,Tamaulipas,38,...,1,2,2,1,4.0,2,5,1,Tamaulipas,3
2349,2023-03-21,1004314,1,80,Sonora,33,1,1,Sonora,33,...,2,2,2,1,4.0,2,5,1,Sonora,33
6269,2023-03-21,1001845,1,11,Nayarit,11,2,2,Nayarit,17,...,2,2,2,1,4.0,1,2,2,Nayarit,11
7763,2023-03-21,1001943,2,45,Quintana Roo,5,2,2,Quintana Roo,5,...,2,2,2,1,1.0,1,5,2,Quintana Roo,5
8133,2023-03-21,1011234,2,24,Quintana Roo,5,2,2,Quintana Roo,5,...,2,2,2,1,4.0,1,5,2,Quintana Roo,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118770,2020-12-30,880573,1,11,Sinaloa,16,2,2,Sinaloa,12,...,2,2,2,1,4.0,1,5,1,Sinaloa,16
119191,2020-12-30,883479,2,40,Jalisco,97,2,2,Jalisco,97,...,2,2,2,1,4.0,1,5,1,Jalisco,97
119213,2020-12-30,884032,1,29,Guerrero,38,2,2,Guerrero,38,...,2,2,2,1,4.0,2,5,1,Guerrero,38
119882,2020-12-30,883941,1,54,Jalisco,97,2,2,Jalisco,97,...,2,2,2,1,4.0,1,5,1,Jalisco,97


In [40]:
dengue['DICTAMEN'].unique()

array([4., 1.])

In [41]:
dengue.HEMORRAGICOS.unique()

array([2, 1], dtype=int64)

In [42]:

dengue.ENTIDAD_RES.value_counts(normalize=True)*100

Jalisco                15.625000
Sonora                 11.363636
Oaxaca                  9.375000
Sinaloa                 8.522727
Guerrero                7.102273
Veracruz                6.534091
Morelos                 4.829545
Tamaulipas              4.545455
Tabasco                 4.261364
Michoacán               4.261364
Coahuila                3.977273
Nayarit                 3.409091
Chiapas                 3.409091
San Luis Potosí         2.840909
Puebla                  2.272727
Quintana Roo            1.988636
Colima                  1.988636
Campeche                0.852273
Nuevo León              0.568182
Baja California         0.568182
Yucatán                 0.568182
Ciudad de México        0.284091
Baja California Sur     0.284091
Estado de México        0.284091
Querétaro               0.284091
Name: ENTIDAD_RES, dtype: float64

### Municipios

## ERA5

In [43]:
era5 = pd.read_csv("data/ERA5/ERA5_Land_mean_values_2020_2023.csv")
era5.drop(columns=['system:index','.geo'], inplace=True)
era5.head()

,date,region_id,surface_net_solar_radiation,surface_pressure,temperature_2m,total_precipitation
0,2020-01-01,3,2.134247e+06,98701.231608,287.766367,0.002488
1,2020-01-01,4,3.509537e+06,100301.585572,298.268191,0.000305
2,2020-01-01,5,1.104217e+06,89380.675663,284.446624,0.000748
3,2020-01-01,6,3.107405e+06,95000.606587,294.771764,0.000271
4,2020-01-01,7,3.682448e+06,92870.038564,294.780402,0.000756


In [44]:
era5['region_id'] = era5['region_id'].replace(entidades_federativas)
era5.head()

,date,region_id,surface_net_solar_radiation,surface_pressure,temperature_2m,total_precipitation
0,2020-01-01,Baja California Sur,2.134247e+06,98701.231608,287.766367,0.002488
1,2020-01-01,Campeche,3.509537e+06,100301.585572,298.268191,0.000305
2,2020-01-01,Coahuila,1.104217e+06,89380.675663,284.446624,0.000748
3,2020-01-01,Colima,3.107405e+06,95000.606587,294.771764,0.000271
4,2020-01-01,Chiapas,3.682448e+06,92870.038564,294.780402,0.000756
